<a href="https://colab.research.google.com/github/eunyhwang/TextMining/blob/main/Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests beautifulsoup4

# Web Crawling: Naver News

In [ ]:
import requests
from bs4 import BeautifulSoup

url = f"https://openapi.naver.com/v1/search/news.json?query={'지속가능성'}&display={100}&start={1}&sort={'date'}"
headers = {
    'X-Naver-Client-Id': 'u48m8PJ4OOADnYKmqOaE',
    'X-Naver-Client-Secret': '2wsXFZo3bQ'
}

response = requests.get(url, headers = headers)
#print(response)

soup = BeautifulSoup(response.content, 'html.parser')
title = soup.find('h1').get_text() if soup.find('h1') else 'No Title'
paragraphs = soup.find_all('p')
content = ' '.join([para.get_text() for para in paragraphs])

print({
    'title': title,
    'content': content
})

{'title': 'No Title', 'content': ''}


In [ ]:
### Collecting 1,000 articles with Naver API

import requests
import json
import html # for decoding

client_id = 'u48m8PJ4OOADnYKmqOaE'
client_secret = '2wsXFZo3bQ'

def get_news(query, display=100, start=1, sort='date'):
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display={display}&start={start}&sort={sort}"
    headers = {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def extract_articles(response):
    articles = []
    if response:
        for item in response.get('items', []):
            title = html.unescape(item.get('title'))
            #print('this is title:',title)
            description = html.unescape(item.get('description')) #for decoding
            description = BeautifulSoup(description, 'html.parser').get_text() #for getting only text
            #print('this is description:',description)
            articles.append({'title': title, 'description': description })
    return articles

query = "지속가능성"
articles = []

for start in range(1, 101, 100):
    response = get_news(query, start=start)
    articles.extend(extract_articles(response))

with open('articles.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

print(f"Total articles collected: {len(articles)}")


Total articles collected: 100


In [ ]:
articles[0]['description']

'LG화학 CSSO 이종구 부사장은 "LG화학은 전지 소재, 친환경 지속가능 소재 등 비즈니스 포트폴리오를 지속가능성에 기반하여 구축할 뿐만 아니라, 사용 에너지까지 친환경 에너지로 전환하여 고객들에게 차별화된... '

In [ ]:
import requests
import csv
from bs4 import BeautifulSoup

def scrape_articles(url):
    articles = []

    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all <a> tags within the specified class
        article_links = soup.find('ul', class_='news_list ranking')
        ('news_list ranking a[href]')
        print('this is the links:',article_links)
        # Extract the href attribute from each <a> tag
        for link in article_links:
            article_url = link['href']
            print("this is the url:",article_url)
            # Fetch the full content of the article
            article_content = scrape_article_content(article_url)

            # Create a dictionary representing the article and add it to the list
            articles.append({
                'url': article_url,
                'content': article_content
            })

    return articles

def scrape_article_content(article_url):
    # Send a GET request to the article URL
    response = requests.get(article_url)
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find and extract the content of the article
        # Specifically target the "cont_body" class
        article_content = soup.find('div', class_='cont_body').text.strip()
       #print(article_content)
        return article_content
    else:
        return None

# URL of the Digital Chosun website
base_url = 'https://digitalchosun.dizzo.com/svc/news/ranking.html'
articles = scrape_articles(base_url)


this is the links: []


## BBC Page article crawling

In [ ]:
import requests
import csv
from bs4 import BeautifulSoup

def scrape_articles(url):
  article_hrefs = []
  response = requests.get(url)
  if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    news_lists = soup.find_all('div', class_='promo-text')
    for news_list in news_lists:
      article_links = news_list.find_all('a', href=True)
      for article_link in article_links:
        article_url = article_link['href']
        article_hrefs.append(article_url)

  else:
    print('failed')

  return article_hrefs

def scrape_article_content(article_url):
  response = requests.get(article_url)
  content_text = ''
  if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    article_content = soup.find_all('p', class_='bbc-1uhqor3 e17g058b0')
    for article in article_content:
      content_text += article.get_text(separator='\n').strip() + ('\n')

    return content_text.strip()

  else:
    print('failed')


In [ ]:
url = "https://www.bbc.com/korean/topics/cz74kjy812kt"
article_hrefs = scrape_articles(url)

# Scrape content for each article URL and save to CSV
with open('articles.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['URL', 'Content'])

    for href in article_hrefs:
        content = scrape_article_content(href)
        if content:
          #print("this is href:",href)
          #print("this is content:", content)
          csv_writer.writerow([href, content])
    print("CSV file saved successfully.")

CSV file saved successfully.


## Load csv file

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('articles.csv')
print(df.head())

                                                URL  \
0  https://www.bbc.com/korean/articles/cp667r0gdd5o   
1  https://www.bbc.com/korean/articles/cld0y0qqdq2o   
2  https://www.bbc.com/korean/articles/cxr3d6e3v3xo   
3  https://www.bbc.com/korean/articles/cl4038rez86o   
4  https://www.bbc.com/korean/articles/cz4xw40952do   

                                             Content  
0  지난 21일 영국 런던에서 출발해 싱가포르로 향하던 싱가포르항공 여객기가 심한 난기...  
1  LP를 많이 소비하는 영국에선 이 분야의 탄소 배출량이 사람 400명의 연간 배출량...  
2  미국 콜로라도에 있는 맥주 회사 ‘뉴 벨지움 브루잉’의 시작은 창업자들이 1988년...  
3  지난 16일(현지시간) 하루 동안 아랍에미리트(UAE) 두바이가 기록적인 강우량을 ...  
4  기후변화 때문에 스키 시즌이 점점 불확실해지고 있다. 그런데 최근 과거 인류의 방식...  


In [ ]:
len(df)

20

In [ ]:
article = scrape_articles(url)
article

InvalidURL: Failed to parse: []

In [ ]:
soup = BeautifulSoup.response

In [ ]:
'''
# Save the content of each article into a CSV file
with open('article_contents.csv', 'w', encoding='utf-8', newline='') as csvfile:
    fieldnames = ['link', 'content']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for article in articles:
        if article['content']:
            writer.writerow({'link': article['url'], 'content': article['content']})
'''

In [ ]:
### Get a full content in one channel

import requests
import json
import html
from bs4 import BeautifulSoup

client_id = 'u48m8PJ4OOADnYKmqOaE'
client_secret = '2wsXFZo3bQ'

def get_news(query, display=100, start=1, sort='date'):
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display={display}&start={start}&sort={sort}"
    headers = {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def extract_articles(response):
    articles = []
    if response:
        for item in response.get('items', []):
            title = html.unescape(item.get('title'))
            url = item.get('link')  # Extract the URL of the article
            print('this is url:', url)
            content = get_article_content(url)  # Fetch the full content of the article
            articles.append({'title': title, 'content': content})
            #print(f"Title: {title}")
            #print(f"Content: {content}\n")
    return articles

def get_article_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # Extract the text content of the article
        text_content = soup.get_text(separator='\n')
        return text_content
    else:
        return None

query = "지속가능성"
articles = []

for start in range(1, 101, 100):
    response = get_news(query, start=start)
    articles.extend(extract_articles(response))

# Save articles to a JSON file
with open('articles.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

print(f"Total articles collected: {len(articles)}")


this is url: https://www.econovill.com/news/articleView.html?idxno=656429
this is url: https://n.news.naver.com/mnews/article/469/0000804445?sid=100
this is url: https://www.nbntv.co.kr/news/articleView.html?idxno=3018263
this is url: http://digitalchosun.dizzo.com/site/data/html_dir/2024/05/31/2024053180221.html
this is url: http://news.lghellovision.net/news/articleView.html?idxno=468837
this is url: https://www.ziksir.com/news/articleView.html?idxno=59505
this is url: https://vop.co.kr/A00001654828.html


SSLError: HTTPSConnectionPool(host='vop.co.kr', port=443): Max retries exceeded with url: /A00001654828.html (Caused by SSLError(SSLError(1, '[SSL: DH_KEY_TOO_SMALL] dh key too small (_ssl.c:1007)')))

In [ ]:
articles[0]

{'title': 'LG화학, 국내 민간기업 최대 규모 풍력 재생에너지 확보',
 'description': 'LG화학 CSSO 이종구 부사장은 "LG화학은 전지 소재, 친환경 지속가능 소재 등 비즈니스 포트폴리오를 지속가능성에 기반하여 구축할 뿐만 아니라, 사용 에너지까지 친환경 에너지로 전환하여 고객들에게 차별화된... '}

In [ ]:
import os
import sys
import urllib.request
import pandas as pd
import json
import re

client_id = 'u48m8PJ4OOADnYKmqOaE'
client_secret = '2wsXFZo3bQ'

query = urllib.parse.quote(input("검색어:"))

idx = 0
display = 100
start = 1
end = 1000

url = "https://openai.naver.com/v1/search/webkr?query=" + query

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret", client_secret)
reponse = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
  response_body = response.read()
  print(response_body.decode("utf-8"))

else:
  print("Error Code:" + rescode)

In [ ]:
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStream;
import java.io.InputStreamReader;
import java.net.HttpURLConnection;
import java.net.MalformedURLException;
import java.net.URL;
import java.util.HashMap;
import java.util.Map;

public class ApiExamShortenUrl {

    public static void main(String[] args) {
        String clientId = "YOUR_CLIENT_ID"; //애플리케이션 클라이언트 아이디값"
        String clientSecret = "YOUR_CLIENT_SECRET"; //애플리케이션 클라이언트 시크릿값"

        String originalURL = "https://developers.naver.com/notice";
        String apiURL = "https://openapi.naver.com/v1/util/shorturl?url=" + originalURL;

        Map<String, String> requestHeaders = new HashMap<>();
        requestHeaders.put("X-Naver-Client-Id", clientId);
        requestHeaders.put("X-Naver-Client-Secret", clientSecret);
        String responseBody = get(apiURL,requestHeaders);

        System.out.println(responseBody);
    }

    private static String get(String apiUrl, Map<String, String> requestHeaders){
        HttpURLConnection con = connect(apiUrl);
        try {
            con.setRequestMethod("GET");
            for(Map.Entry<String, String> header :requestHeaders.entrySet()) {
                con.setRequestProperty(header.getKey(), header.getValue());
            }

            int responseCode = con.getResponseCode();
            if (responseCode == HttpURLConnection.HTTP_OK) { // 정상 호출
                return readBody(con.getInputStream());
            } else { // 에러 발생
                return readBody(con.getErrorStream());
            }
        } catch (IOException e) {
            throw new RuntimeException("API 요청과 응답 실패", e);
        } finally {
            con.disconnect();
        }
    }

    private static HttpURLConnection connect(String apiUrl){
        try {
            URL url = new URL(apiUrl);
            return (HttpURLConnection)url.openConnection();
        } catch (MalformedURLException e) {
            throw new RuntimeException("API URL이 잘못되었습니다. : " + apiUrl, e);
        } catch (IOException e) {
            throw new RuntimeException("연결이 실패했습니다. : " + apiUrl, e);
        }
    }

    private static String readBody(InputStream body){
        InputStreamReader streamReader = new InputStreamReader(body);

        try (BufferedReader lineReader = new BufferedReader(streamReader)) {
            StringBuilder responseBody = new StringBuilder();

            String line;
            while ((line = lineReader.readLine()) != null) {
                responseBody.append(line);
            }

            return responseBody.toString();
        } catch (IOException e) {
            throw new RuntimeException("API 응답을 읽는데 실패했습니다.", e);
        }
    }
}

In [ ]:
# version of csv
import requests
from bs4 import BeautifulSoup
import csv

# URL of the webpage
url = "https://www.bbc.com/korean/topics/cz74kjy812kt"

# Send an HTTP request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the webpage
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all elements with the specified class
    articles = soup.find_all('a', class_='focusIndicatorDisplayBlock bbc-uk8dsi e1d658bg0')

    # Open a CSV file to save the content
    with open("article_links.csv", "w", newline='', encoding="utf-8") as file:
        writer = csv.writer(file)
        # Write the header row
        writer.writerow(["Title", "Link"])

        # Loop through each article element and extract the text and href
        for article in articles:
            title = article.get_text(strip=True)
            link = f"https://www.bbc.com{article['href']}"
            # Write the title and link to the CSV file
            writer.writerow([title, link])

    print("Article links and titles saved to article_links.csv")
else:
    print("Failed to retrieve the webpage")

Article links and titles saved to article_links.csv
